# Image Contains API
Training a computer vision model to detect people in images? Have petabytes of surveillance footage and want to know which frames  need to be inspected? These are just some of the common use cases for detecting whether images contain something.

To build an effective machine learning model to recognize things in images requires a large amount of high quality training data. Customers have expressed they want to get training data using Amazon Mechanical Turk (MTurk) quickly and easily without having to be familiar with crowdsourcing, coding, or spending hours on experimentation. 

With this preview Image Contains API, we have greatly simplified the steps needed to get results from MTurk. The API takes as input an image URL and a type of thing, then returns true or false if that thing is in the image.

For this project we will be asking Workers if this image contains a dog. 

| Image | Label |
| --- | --- |
| ![Butterbean Image](https://www.mturk.com/media/butterbean.jpg) | dog |

Input: `{"image": {"url": "https://www.mturk.com/media/butterbean.jpg"}, "target": {"label": "dog"} }`

Since the image cointains a dog, it should return true.

Result: `{"containsTarget": true}`

The API implements an adjudication strategy based on Worker agreement and returns results only when it reaches confidence. MTurk has provided a Python client along with sample code so that customers can quickly get started, including examples on how to call the API for many inputs.

### Prerequisites

Check out [Chapter 0 Getting Started](Chapter 0.ipynb) to configure your notebook if you've not done so already. 

## First we will import the packages we need to use the API

In [2]:
from mturk_crowd_beta_client import MTurkCrowdClient
from boto3.session import Session
import uuid

## Next we set up a session

This examples assume you have a local AWS profile called
 
__'mturk-crowd-caller'__, but you can authenticate however you like,
 
including by directly passing in your access key and secret key.

In [ ]:
session = Session(profile_name='mturk-crowd-caller')

### Create the client Session


In [ ]:
crowd_client = MTurkCrowdClient(session)

## Create a task_name

For this example, we'll give our task a random, unique name. 

However you probably want to pick a name based on your input source when you use in production

In [5]:
task_name = 'my-test-task-' + uuid.uuid4().hex

## Identify the API you will use


Next, we specify the name of the api to call

In [6]:
function_name = 'image-contains'

## Speficy Input

The Image Contains API takes as input an image url and a text label. Here we will assign them to a variable to make it eaiser to see how to call the API.

```
"input": {
    "image": {"url": "<image you want analyzed>"},
    "target": {"label": "<the type of thing you're looking for>"}
  }
  ```

### The image we want labeled

In [7]:
image_url = 'https://www.mturk.com/media/butterbean.jpg'

### The type of thing we're looking for

In [8]:
label = 'dog'

# Create the task and print it out so you can see it.

Tasks are created by calling PUT. You will need to pass in the API name as function_name, unique task name, and input. 

Below we've assembled the put_task for you and printed out the response. 

__status code__ should be 201 if successful. 

__task__ will display your input, taskName, problemDetails, state, and result.   

*__Note:__ Runing this will send your task to actual MTurk Workers, so If you didn’t prepay for HITs in your MTurk Requester account during account setup, you’ll get an InsufficientFunds error from the prod API.*

Please visit [this page](https://requester.mturk.com/prepayments/new) to prepay in your MTurk account. There’s a $1 minimum. 

For more details on pricing, go to [Paying for Work from a Preview API](https://medium.com/@mechanicalturk/paying-for-work-from-preview-api-481ab24da26d)

In [ ]:
put_result = crowd_client.put_task(function_name,task_name,{'image': {'url': image_url}, 'target': {'label': label} })

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))
        


*Alternativly if you want to call it all in one line you can write your API call like this:*

In [ ]:
client.put_task('image-contains', 'my-task-name', {'image': {'url': 'https://www.mturk.com/media/butterbean.jpg'}, 'target': {'label': 'dog'})

## Retrive Results

After creating a task, users can call the GET operation to poll its current state. For more details about PUT or GET, check out [Instructions for calling a Preview API](https://medium.com/@mechanicalturk/instructions-for-calling-a-preview-api-620f572bedc4#3753) 

Get the task we just created. Note that since this is a prod (i.e., non-test) task, we'll have to poll periodically until the task completed.

When the task is still in progress, we'll see 'state': 'processing' in the response.

If successfull __status_code__ will return as 200.

Once Workers have completed the task, you will see a result like ` "containsTarget": true`

Remember this is going to real Workers, so if your task is still in progress, wait a few minutes and call get_task again. 

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(
{'status_code': get_result.status_code, 'task': get_result.json()}))

## Additional Support
If you have any questions or feedback, such as methods you wish our client supported, [please contact our product team](mailto:mturk-requester-preview@amazon.com), chat with us in our [Slack channel](https://amzn-mturk.slack.com/) by joining via [this link](https://amzn-mturk.slack.com/join/shared_invite/MjM4MzczOTI5MDQ3LTE1MDQ3MzU0MTItMzhlMTg2OWRhNA), or submit a pull request [on GitHub](https://github.com/awslabs/mturk-crowd-beta-client-python) adding additional functionality to our client.